In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import pandas as pd

In [3]:
df=pd.DataFrame(
{
    'a':[1,1,2,3,4],
    'b':[2,2,3,2,1],
    'c':[4,6,7,8,9],
    'd':[4,3,4,5,4]
})

X=df.assign(const=1)
X

,a,b,c,d,const
0,1,2,4,4,1
1,1,2,6,3,1
2,2,3,7,4,1
3,3,2,8,5,1
4,4,1,9,4,1


In [13]:
pd.Series([vif(X.values,i) for i in range(X.shape[1])],index=X.columns)

b         1.185185
c         1.370370
d         1.185185
const    61.666667
dtype: float64

In [8]:
X.drop('a',axis=1,inplace=True)
pd.Series([vif(X.values,i) for i in range(X.shape[1])],index=X.columns)

b         1.185185
c         1.370370
d         1.185185
const    61.666667
dtype: float64

In [10]:
import numpy as np
import statsmodels.formula.api as smf

def get_vif(exogs,data):
    vif_array=np.array([])
    tolerance_array=np.array([])
    
    for exog in exogs:
        not_exog=[i for i in exogs if i!=exog]
        formula=f"{exog} ~ {' + '.join(not_exog)}"
        
        r_squared=smf.ols(formula,data=data).fit().rsquared
        
        vif=1/(1-r_squared)
        vif_array=np.append(vif_array,vif).round(2)
        
        tolerance=1-r_squared
        tolerance_array=np.append(tolerance_array,tolerance).round(2)
    df_vif=pd.DataFrame({'VIF':vif_array,'Tolerance':tolerance_array},index=exogs)
    
    return df_vif

In [11]:
get_vif(exogs=df.columns,data=df)

,VIF,Tolerance
a,22.95,0.04
b,3.00,0.33
c,12.95,0.08
d,3.00,0.33


# Thresholds
1. VIF - If it is higher then 10, it is a problem. Remove the columns with highest VIF and test again
2. Tolerance - If it is lower then 0.1, it is a problem.